# Imports

In [ ]:
import torch
from torch import nn
from torch.nn.functional import cross_entropy, softmax, binary_cross_entropy_with_logits

# Debug WBCE-by-disease

In [ ]:
%run ./wbce.py

In [ ]:
loss = WeigthedBCEByDiseaseLoss()

In [ ]:
bs = 5
out = torch.randn(bs, 4)
target = (torch.rand(bs, 4) > 0.75).long()
out, torch.sigmoid(out), target

In [ ]:
loss(out, target)

## BCE with weights

In [ ]:
bs = 3
out = torch.randn(bs, 4)
target = (torch.rand(bs, 4) > 0.75).long()
out, torch.sigmoid(out), target

In [ ]:
loss = nn.BCEWithLogitsLoss(reduction='none')
loss(out, target.float())

In [ ]:
loss.to('cuda')

In [ ]:
loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1, 2, 3, 4]), reduction='none')
loss(out, target.float())

# Debug Focal loss

In [ ]:
%run focal.py

In [ ]:
alpha = 0.75
gamma = 2

## Multilabel case

In [ ]:
outputs = torch.tensor([[1, 1, 0, 0, 0, 1],
                        [0, 0, 1, 0, 0, 1],
                       ]).float()
targets = torch.tensor([[1, 1, 1, 0, 1, 0],
                        [0, 1, 0, 0, 1, 0],
                       ]).float()
outputs

In [ ]:
focal = FocalLoss(alpha=0.87, reduction='none')
focal(outputs, targets)

## Multiclass case

In [ ]:
outputs = torch.tensor([[0, 1, 0],
                        [-3, 1, 2],
                        [1, 11.1, 0.5],
                       ]).float()
targets = torch.tensor([1, 2, 0]).long()

In [ ]:
focal = FocalLoss(alpha=0.9, gamma=2, multilabel=False, reduction='none')
focal(outputs, targets)